<h3>Optimal Trading</h3>
<p> Instead of doing simple maximum and minima, we use an algorithm below which calculates the most optimal times to buy and sell and then we use this for the models that we have </p>

In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('../../../')
print(os.getcwd())
from v2.model import Trading
import plotly.graph_objs as go
from plotly.offline import plot
from scipy.signal import argrelextrema
from tensorflow.keras.models import load_model
import pickle
from v2.strategy.indicators.param import Param
from v2.strategy.indicators.rsi import RSI
from v2.strategy.indicators.macd import MACD
from v2.strategy.indicators.mmroc import MinMaxRateOfChange
from v2.strategy.indicators.stochastic_oscillator import StochasticOscillator

/Users/rosscopeland/Desktop/personal/code/vivaldi/back_testing


Load the Trading configuration and instantiate a Trading model

In [2]:
def load_config():
    my_config = {}
    with open('config.config') as config:
        for line in config:
            args = line.split('=')
            my_config[args[0]] = args[1].rstrip().split(',')
    return my_config

model = Trading(load_config())

Compute optimal times to buy/sell for each dataset:

In [3]:
#this only works on one dataset at a time
candle, buys, sells = None, None, None
datasets = model.dfs
fee = 0.0026
model_trough = load_model('v2/strategy/saved_models/boost_rf_lstm.sav')
model_boost = pickle.load(open('v2/strategy/saved_models/lstm_trough/part_boost_lstm.sav', 'rb'))
model_rf = pickle.load(open('v2/strategy/saved_models/lstm_trough/part_rf_lstm.sav', 'rb'))
for d in datasets:
    ## indicator setup
    dataset = d[0]

    stoch_highlow = Param(5, 10000, 0, 'highlow_range', 90.0)
    stoch_k = Param(5, 10000, 0, 'k_period', 270.0)
    stoch_oscillator = StochasticOscillator(_params=[stoch_highlow, stoch_k], _name='stochastic_oscillator')
    stoch_oscillator.genData(dataset, gen_new_values=False)

    ema_fast = Param(5, 10000, 0, 'ema_fast', 60)
    ema_slow= Param(6, 10001, 0, 'ema_slow', 120)
    signal = Param(5, 10001, 0, 'signal', 90)
    macd_ = MACD(_params=[ema_fast, ema_slow, signal], _name='macd')
    macd_.genData(dataset, gen_new_values=False)

    rsi_period = Param(5, 10000, 0, 'period', 90.0)
    rsi_ = RSI(_params=[rsi_period], _name='rsi')
    rsi_.genData(dataset, gen_new_values=False)

    mmroc_period = Param(5, 10001, 0, 'period', 30)
    mmroc = MinMaxRateOfChange(_params=[mmroc_period], _name='mmroc')
    mmroc.genData(dataset, gen_new_values=False)

    buy_prices = []
    buy_times = []
    sell_prices = []
    sell_times = []
    prediction_times = []
    prediction_prices = []
    name = d[1]
    cur_min = 99999999999999999.00
    cur_max = 0.00
    cur_min_time = 00.0
    cur_max_time = 00.0
    last_buy_price = 000.00
    last_buy_time = 000.00
    sell_time = 0000.0
    
    looking_for_buy = True

    for row in dataset.itertuples():

        close = row.close
        time = row.time
        ## model stuff
        rf_model_data = np.array([row.stosc_k, row.mmroc, row.macd_diff, row.rsi])
        
        if not np.isnan(np.sum(rf_model_data)):
            rf_model_data = rf_model_data.reshape(1, -1)
            rf_model_prediction = model_rf.predict(rf_model_data)[0]
        else:
            rf_model_prediction = 0.0

        boost_model_data = np.array([row.stosc_k, row.mmroc, row.macd_diff, row.rsi, rf_model_prediction])
        if not np.isnan(np.sum(boost_model_data)):
            boost_model_data = boost_model_data.reshape(1, -1)
            boost_model_prediction = model_boost.predict(boost_model_data)[0]
        else:
            boost_model_prediction = 0.0

        nn_model_data = np.array([row.stosc_k, row.mmroc, row.macd_diff, row.rsi, rf_model_prediction, boost_model_prediction])
        if not np.isnan(np.sum(nn_model_data)):
            nn_model_data = nn_model_data.reshape(1, -1)
            nn_model_data = np.reshape(nn_model_data, (nn_model_data.shape[0], 1, nn_model_data.shape[1]))
            prediction = model_trough.predict(nn_model_data)[0][1]
        else:
            prediction = 0.0
        if prediction > 0.85:
            prediction_prices.append(close)
            prediction_times.append(time)


        ## optimal stuff
        
        if looking_for_buy:
            if close < cur_min:
                cur_min = close
                cur_min_time = time
                cur_max = close
                cur_max_time = time
            elif close > cur_max:
                cur_max = close
                cur_max_time = time
                delta = cur_max - cur_min
                if delta > ((cur_min * fee) + (cur_max * fee)):
                    looking_for_buy = False
                    last_buy_price = cur_min
                    last_buy_time = cur_min_time
        else:
            trade_fee = ((cur_min * fee) + (cur_max * fee))
            if close > cur_max:
                cur_max = close
                cur_max_time = time
            elif close < cur_max - trade_fee:
                looking_for_buy = True
                buy_prices.append(cur_min)
                buy_times.append(cur_min_time)
                sell_prices.append(cur_max)
                sell_times.append(cur_max_time)
                cur_min = close
                cur_min_time = time
                cur_max = close
                cur_max_time = time

    candle = go.Candlestick(x=dataset['time'], open=dataset['open'], close=dataset['close'], high=dataset['high'], low=dataset['low'], name='Candlesticks')

    buys = go.Scatter(x=buy_times, y=buy_prices, name='Optimal Entries', mode='markers')
    sells = go.Scatter(x=sell_times, y=sell_prices, name='Optimal Exits', mode='markers')
    predicts = go.Scatter(x=prediction_times, y=prediction_prices,  name='ML Entries', mode='markers')
    dataset['trough'] = dataset.iloc[argrelextrema(dataset.close.values, np.less_equal, order=480)[0]]['close']
    dataset['trough'] = dataset['trough'].fillna(0)
    trough_values = []
    trough_times = []
    for x in dataset[['time', 'trough']].itertuples():
        if x.trough:
            trough_times.append(x.time)
            trough_values.append(x.trough)
    troughs = go.Scatter(x=trough_times, y=trough_values, name='Troughs (Local Maximums)', mode='markers')
    data = [buys] + [sells] + [candle] + [troughs] + [predicts]

Plot optimal points to buy/sell

In [4]:
layout = go.Layout(title='Optimal Buying Times/Detected Troughs for ' + name)
fig = go.Figure(data=data, layout=layout)
plot(fig, filename='plots/optimal_' + name + '.html')

'plots/optimal_XBTUSD.html'